# Imports

In [1]:
from sklearn.model_selection import StratifiedKFold
from attention import Attention
from utilities import *
from keras.models import Model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import pandas as pd

n_splits = 5
X = np.load('data/X.npy')
y = np.load('data/y.npy')
splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2019).split(X, y))

# change data dir based on machine
local = True
if local is True:
    data_dir = 'data/'
else:
    data_dir = '../input/vsb-power-line-fault-detection/'

Using TensorFlow backend.


## Functions

In [32]:
def lstm_model(input_shape):

    # input layer then attach attention
    inp = Input(shape=(input_shape[1], input_shape[2],))

    x = Bidirectional(LSTM(128, return_sequences=True))(inp)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)

    x = Attention(input_shape[1])(x)

    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[matthews_correlation]) 

    return model

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [33]:
model = lstm_model(X_train.shape)
print(model.metrics_names)
model.summary()

['loss', 'matthews_correlation']
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 160, 57)           0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 160, 256)          190464    
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 160, 128)          164352    
_________________________________________________________________
attention_6 (Attention)      (None, 128)               288       
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 363,425
Trainable params: 363,425
Non-trainable params: 0
____________________

In [34]:
ckp = ModelCheckpoint('training_weights.h5', save_weights_only=True, verbose=1, monitor='val_loss', mode='max')
model.fit(X_train, y_train, batch_size=100, epochs=2, validation_data=[X_valid, y_valid], callbacks=[ckp])

Train on 2323 samples, validate on 581 samples
Epoch 1/2
2323/2323 [==============================] - 41s 18ms/step - loss: 0.3190 - matthews_correlation: 0.0000e+00 - val_loss: 0.2081 - val_matthews_correlation: 0.0000e+00

Epoch 00001: saving model to training_weights.h5
Epoch 2/2
2323/2323 [==============================] - 43s 19ms/step - loss: 0.2325 - matthews_correlation: 0.0000e+00 - val_loss: 0.1879 - val_matthews_correlation: 0.0000e+00

Epoch 00002: saving model to training_weights.h5


In [35]:
# save the model
model.save('attention_wo_cval_short.h5')

In [20]:
meta_test = pd.read_csv(data_dir + 'metadata_test.csv')
meta_test = meta_test.set_index(['signal_id'])
meta_test.head()

,id_measurement,phase
signal_id,,
8712,2904,0
8713,2904,1
8714,2904,2
8715,2905,0
8716,2905,1


In [24]:
x_test_input = np.load(data_dir + 'x_test.npy')
x_test_input.shape

(6779, 160, 57)

# Loading preexisting model

In [37]:
model = load_model('attention_wo_cval_short.h5', custom_objects={'Attention': Attention, 'matthews_correlation': matthews_correlation})

In [39]:
model.metrics_names

['loss', 'matthews_correlation']

In [ ]:
model.load_weights()

# Make submission file

In [41]:
preds = np.load('predictions_fromcrossval.npy')
np.unique(preds, return_counts=True)

(array([6.2602758e-04, 8.2722306e-04, 8.3577633e-04, ..., 9.2560065e-01,
        9.2668343e-01, 9.2811823e-01], dtype=float32),
 array([3, 3, 3, ..., 3, 3, 3], dtype=int64))

In [47]:
best_thres = 0.36
preds_new = (np.squeeze(np.mean(preds, axis=0)) > best_thres).astype(np.int)
np.unique(preds_new, return_counts=True)

(array([0, 1]), array([19506,   831], dtype=int64))

In [48]:
preds_new.to_csv('final_predictions', index=False)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [24]:
submission = pd.read_csv(data_dir + 'sample_submission.csv')
print(len(submission))
submission.head()

20337


,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [61]:
# write submission file
submission['target'] = preds_new
submission['target'].value_counts()
#submission.to_csv('submission.csv', index=False)

0    19506
1      831
Name: target, dtype: int64